# Improving RAG by Averaging

#### Authors: Gilyoung Cheong, Qidu Fu, Junichi Koganemaru, Xinyuan Lai, Sixuan Lou, Dapeng Shang
 
This notebook is written as a part of the capstone project for the [Erdős Institute Data Science Boot Camp](https://www.erdosinstitute.org/). The data used in this notebook is provided by Jason Morgan at AwareHQ. We use Gemma 2B-IT using HuggingFace API, which we learned from [this article](https://huggingface.co/learn/cookbook/en/rag_with_hugging_face_gemma_mongodb) by Richmond Alake.

In this notebook, we implement some pipelines of [Retrieval-Augmented Generation (RAG)](https://aws.amazon.com/what-is/retrieval-augmented-generation/) using [SBERT](https://arxiv.org/abs/1908.10084) developed by Nils Reimers and Iryna Gurevych. The documentation for the SBERT API for Python is available in [this link](https://sbert.net/). We use SBERT to find relevant comments to a query from various reddit comments from previously saved data.

The pretrained SBERT converts any sentence into a vector in $\mathbb{R}^{1024}$, and the relevance of the two sentences is simply measured by the cosine similarity of the corresponding vectors. That is, if $\boldsymbol{u}$ and $\boldsymbol{v}$ are the vectors, we measure 

$$\frac{\langle \boldsymbol{u}, \boldsymbol{v} \rangle}{\|\boldsymbol{u}\|\|\boldsymbol{v}\|},$$

which can be intuitively thought as $\cos(\theta)$, where $\theta$ is the angle between $\boldsymbol{u}$ and $\boldsymbol{v}$.

## Benefits of SBERT vs BERT

SBERT (Sentence Bert) is based on [BERT (Bidirectional Encoder Representations from Transformer)](https://arxiv.org/abs/1810.04805) developed by Google. From inspection, there are clear benefits of using SBERT over BERT for our purpose.

1. BERT is designed to generate vectors that correspond to individual words (or more precisely, *subwords*) to a sentence, so each sentence is converted into not just a vector but a sequence of vectors. Hence, in order to examine the similaritiy of two sentences, we need to either pick one word or take the average of the vectors, which did not yield satisfying results.

2. Because BERT converts every subword as a vector, in order to fully use it, we need to use a lot more storage. In an experiement, examining 10400 comments required 11.8GB with BERT while it only required 91.6MB with SBERT.

3. For BERT, the query and the comments (i.e., information to answer the query) need to be proceeded together when we embedd them as (sequences of) vectors. For SBERT, we can vectorize the comments first and then indepedently vectorize the query later.

In [2]:
# Importing necessary libraries 
import numpy as np
import pandas as pd
import torch
import re
import nltk
import os.path

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm
from sentence_transformers import SentenceTransformer # SBERT
from sklearn.metrics.pairwise import cosine_similarity
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM # HuggingFace API to use Gemma (LLM)
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [4]:
#Defining model for synthetic query generation
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')

## 1. Loading and cleaning the data

In [5]:
#Reading in reddit data and specifying columns to use
columns = ['reddit_text', 'reddit_subreddit']
df = pd.read_parquet("reddit.parquet", engine='fastparquet', columns = columns)

In [6]:
#Identifying candidate subreddits to analyze 
value_count = df['reddit_subreddit'].value_counts()
value_count

reddit_subreddit
nursing                789499
walmart                630962
sysadmin               557558
starbucks              393597
WaltDisneyWorld        373138
Target                 340401
UPSers                 262483
Disneyland             231981
Lowes                  198805
CVS                    179598
McDonaldsEmployees     174679
cybersecurity          161868
Fedexers               154572
GameStop               137071
starbucksbaristas      132019
fidelityinvestments    129423
Bestbuy                121077
wholefoods              82052
Panera                  79436
DisneyWorld             65549
DollarTree              59745
TjMaxx                  46286
disney                  43954
McLounge                38627
GeneralMotors           37277
TalesFromYourBank       28444
cabincrewcareers        23408
Chase                   16931
KrakenSupport           14533
WalmartEmployees        10752
BestBuyWorkers           5629
RiteAid                  3970
PaneraEmployees        

In [7]:
#For proof of concept, we decided to work with medium sized subreddits first
value_count[ value_count < 100000]

reddit_subreddit
wholefoods           82052
Panera               79436
DisneyWorld          65549
DollarTree           59745
TjMaxx               46286
disney               43954
McLounge             38627
GeneralMotors        37277
TalesFromYourBank    28444
cabincrewcareers     23408
Chase                16931
KrakenSupport        14533
WalmartEmployees     10752
BestBuyWorkers        5629
RiteAid               3970
PaneraEmployees       2694
FedEmployees           280
Name: count, dtype: int64

In [8]:
selected_subreddts = ['TalesFromYourBank', 'cabincrewcareers', 'Chase', 'KrakenSupport', 'WalmartEmployees', 'BestBuyWorkers', 'RiteAid', 'PaneraEmployees', 'FedEmployees']

In [9]:
df = df[df['reddit_subreddit'].isin(selected_subreddts)]
len(df)

106641

In [10]:
#Basic data cleaning by removing rows with empty/removed/deleted texts
def remove_empty(df):
    df = df[~(df['reddit_text'] == '')] # erasing empty reddit texts
    df = df[~(df['reddit_text']=='[removed]')] # erasing removed reddit texts
    df = df[~(df['reddit_text']=='[deleted]')] # erasing deleted reddit texts
    df = df[df['reddit_text'].str.len() > 5] #Only kee ping responses longer than 5 characters
    df = df.sort_values(by='reddit_text') # sort them by reddit texts
    df = df.reset_index().drop(columns='index') # resetting indices
    return df

In [11]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www.\.\S+')
    return pattern.sub(r'', text)

In [12]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize WordNetLemmatizer
LEMMATIZER = WordNetLemmatizer()

# Load stopwords
STOP_WORDS = set(stopwords.words('english'))

def preprocess_text(text):
    if text is None:
        return None
    text = text.replace('\n', '')
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# from: https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

chat_words_str = """
afaik=As far as i know
afk=away from keyboard
asap=as soon as possible
atk=at the keyboard
atm=at the moment
a3=anytime, anywhere, anyplace
bak=back at keyboard
bbl=be back later
bbs=be back soon
bfn=bye for now
b4n=bye for now
brb=be right back
brt=be right there
btw=by the way
b4=before
b4n=bye for now
cu=see you
cul8r=see you later
cya=see you
faq=frequently asked questions
fc=fingers crossed
fwiw=for what it's worth
fyi=for your information
gal=get a life
gg=good game
gn=good night
gmta=great minds think alike
gr8=great!
g9=genius
ic=I see
icq=I seek you (also a chat program)
ilu=ilu: I love you
imho=in my honest/humble opinion
imo=in my opinion
iow=in other words
irl=in real life
kiss=keep it simple, stupid
ldr=long distance relationship
lmao=laugh my a.. off
lol=laughing out loud
ltns=long time no see
l8r=later
mte=my thoughts exactly
m8=mate
nrn=no reply necessary
oic=oh I see
pita=pain in the a..
prt=party
prw=parents are watching
rofl=rolling on the floor laughing
roflol=rolling on the floor laughing out loud
rotflmao=rolling on the floor laughing my a.. off
sk8=skate
stats=your sex and age
asl=age, sex, location
thx=thank you
ttfn=ta-ta for now!
ttyl=talk to you later
u=you
u2=you too
u4e=yours for ever
wb=welcome back
wtf=what the f...
wtg=way to go!
wuf=where are you from?
w8=wait...
7k=sick:-d laugher
"""

<>:3: SyntaxWarning: invalid escape sequence '\)'
<>:4: SyntaxWarning: invalid escape sequence '\)'
<>:5: SyntaxWarning: invalid escape sequence '\]'
<>:6: SyntaxWarning: invalid escape sequence '\]'
<>:11: SyntaxWarning: invalid escape sequence '\)'
<>:12: SyntaxWarning: invalid escape sequence '\)'
<>:13: SyntaxWarning: invalid escape sequence '\}'
<>:14: SyntaxWarning: invalid escape sequence '\}'
<>:15: SyntaxWarning: invalid escape sequence '\)'
<>:16: SyntaxWarning: invalid escape sequence '\)'
<>:17: SyntaxWarning: invalid escape sequence '\^'
<>:18: SyntaxWarning: invalid escape sequence '\]'
<>:19: SyntaxWarning: invalid escape sequence '\)'
<>:28: SyntaxWarning: invalid escape sequence '\^'
<>:29: SyntaxWarning: invalid escape sequence '\)'
<>:30: SyntaxWarning: invalid escape sequence '\('
<>:31: SyntaxWarning: invalid escape sequence '\('
<>:32: SyntaxWarning: invalid escape sequence '\('
<>:37: SyntaxWarning: invalid escape sequence '\['
<>:38: SyntaxWarning: invalid escap

In [14]:
#Remove emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

#Remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [15]:
#Chat word conversion
def build_chat_word_mapping(chat_words_str):
    mapping = {}
    cw_set = set()
    for line in chat_words_str.split('\n'):
        if line != '':
            cw, cw_description = line.split('=')[0], line.split('=')[1]
            cw_set.add(cw)
            mapping[cw] = cw_description
    return mapping, cw_set

CW_MAPPING, CW_SET = build_chat_word_mapping(chat_words_str)

def convert_chat_word(text):
    new_text = []
    for word in text.split():
        if word.upper() in CW_SET:
              new_text.append(CW_MAPPING[word.upper()])
        else:
              new_text.append(word)
    return ' '.join(new_text)

In [16]:
#Applying cleaning methods.
tqdm.pandas()
df = remove_empty(df)
df['reddit_text']= df['reddit_text'].progress_apply(remove_url) 
df['reddit_text']= df['reddit_text'].progress_apply(preprocess_text) 
df['reddit_text']= df['reddit_text'].progress_apply(remove_emoticons)
df['reddit_text']= df['reddit_text'].progress_apply(remove_emoji)
df['reddit_text']= df['reddit_text'].progress_apply(convert_chat_word)
df = remove_empty(df)

100%|██████████| 102876/102876 [00:00<00:00, 140780.91it/s]


In [18]:
df.sample(20)

,reddit_text,reddit_subreddit
98823,i did! i got a CJO,cabincrewcareers
27093,How'd it go?,Chase
18322,Good question. Work on that resume just in cas...,RiteAid
34887,I know someone aged 55 who got a job offer thi...,cabincrewcareers
38421,I understand that question if you work in a ca...,TalesFromYourBank
58329,"No, do not use your account number for anythin...",TalesFromYourBank
51432,Just a simple question for me to clarify. Can ...,KrakenSupport
61105,"Oh, u got fired from Frontier- I am sure there...",cabincrewcareers
29235,I checked my support history. It shows no acti...,KrakenSupport
100758,so funny you say this when last night the part...,cabincrewcareers


In [19]:
#Saving cleaned data for potential future use
df.to_csv('processeddata.csv', index=True)

## 2. Using SBERT to rank the reddit comments

In [20]:
#Defining SBERT model for generating sentence embeddings
sentence_model = SentenceTransformer("thenlper/gte-large")

def get_sentence_embedding(text):
    if not text.strip(): 
    #.strip() gets rid of new lines
        print("Attempted to get embedding for empty text.")
        return []

    embedding = sentence_model.encode(text)

    return embedding.tolist()

def cos_angle(v, w): 
#inputs v, w are vectors in 1-dimensional tensors (np.array(list))
    v = v.reshape(1,-1)
    w = w.reshape(1,-1)
    return cosine_similarity(v, w)

def inn(v, w):
    return v @ w

In [21]:
# loading or generating embeddings
embedding_results = []

for col in tqdm(selected_subreddts, position=1):
    dfcol = df[df['reddit_subreddit']==col]
    dfcol = dfcol.sort_values(by='reddit_text') # sort them by reddit texts
    dfcol = dfcol.reset_index().drop(columns='index') # resetting indices
    if os.path.exists(col+'_embeddings.pt'):
        textembedding = torch.load(col + '_embeddings.pt')
        dfcol = dfcol.assign(textembedding = textembedding)
    else:
        textembedding = [get_sentence_embedding(x) for x in tqdm(dfcol['reddit_text'], position=0)] #compute sentence embeddings
        torch.save(textembedding, col + '_embeddings.pt') #save embeddings for potential use later
        dfcol = dfcol.assign(textembedding = textembedding)
        dfcol.to_csv(col+'.csv', index=True) #save dataframe for potential use later
    
    d = {
    "subreddit": col,
    "dataframe": dfcol,
    "average_embedding": np.average(textembedding,axis = 0)}
    
    embedding_results.append(d)
    
db = pd.DataFrame(embedding_results)

100%|██████████| 9/9 [00:10<00:00,  1.17s/it]


In [22]:
db.head(10)

,subreddit,dataframe,average_embedding
0,TalesFromYourBank,r...,"[-0.005197360952537699, -0.006250292531688477,..."
1,cabincrewcareers,r...,"[0.00043171990228924506, -0.005137783052966276..."
2,Chase,r...,"[-0.011372712559842126, -0.0010401233618315545..."
3,KrakenSupport,r...,"[-0.0017264996006939439, -0.001783643988806693..."
4,WalmartEmployees,r...,"[-0.008500097343943296, -0.00248342203242274, ..."
5,BestBuyWorkers,re...,"[-0.005873798435248916, -0.0051915133856321564..."
6,RiteAid,re...,"[-0.0054854856773563125, -0.002291048301678379..."
7,PaneraEmployees,re...,"[-0.006839217325948276, -0.0024444137202842757..."
8,FedEmployees,red...,"[-0.0045604230282108235, -0.006909224058766821..."


In [23]:
db.to_csv('data.csv')

In [24]:
clusterered_embeddings = []

num_clusters = 4
clustering_model = KMeans(n_clusters=num_clusters, random_state=0)

for col in tqdm(selected_subreddts):
    dfcol = df[df['reddit_subreddit']==col]
    dfcol = dfcol.sort_values(by='reddit_text') #sort them by reddit texts
    dfcol = dfcol.reset_index().drop(columns='index') #resetting indices
    
    textembedding = torch.load(col + '_embeddings.pt')
    dfcol = dfcol.assign(textembedding = textembedding) #create 'textembedding' column and store textembedding in it

    X = preprocessing.normalize(textembedding)
    clustering_model.fit(X)
    dfcol['Cluster ID'] = pd.Series(clustering_model.fit_predict(X))
    
    for i in range(num_clusters):
        temp = dfcol[dfcol['Cluster ID'] == i]
        d = {
        "subreddit": col + '-Cluster' + str(i),
        "dataframe": temp,
        "average_embedding": np.average([x for x in temp['textembedding']], axis = 0)}
        clusterered_embeddings.append(d)
    
    
db_cluster = pd.DataFrame(clusterered_embeddings)

100%|██████████| 9/9 [00:27<00:00,  3.00s/it]


In [25]:
db_cluster.head(10)

,subreddit,dataframe,average_embedding
0,TalesFromYourBank-Cluster0,r...,"[-0.0040272078714856715, -0.003254711372103259..."
1,TalesFromYourBank-Cluster1,r...,"[-0.003341803076366174, -0.007956792389939725,..."
2,TalesFromYourBank-Cluster2,r...,"[-0.006795902141620346, -0.005613458560315768,..."
3,TalesFromYourBank-Cluster3,r...,"[-0.005708994377133483, -0.009339750569399848,..."
4,cabincrewcareers-Cluster0,r...,"[-0.008814253972035405, 0.0008850114829782974,..."
5,cabincrewcareers-Cluster1,r...,"[-0.0012390267438731216, -0.010243365836453102..."
6,cabincrewcareers-Cluster2,r...,"[-0.002012449527727655, -0.0032770080859143846..."
7,cabincrewcareers-Cluster3,r...,"[0.008617801835901648, -0.005259758658647718, ..."
8,Chase-Cluster0,r...,"[-0.00861372841124063, -0.0026734423179895425,..."
9,Chase-Cluster1,r...,"[-0.013078588633965708, -0.004586292042826056,..."


In [29]:
print("Total number of clusters: " + str(len(db_cluster)))

Total number of clusters: 36


In [30]:
db_cluster.to_csv('clustereddata.csv')

In [31]:
query = "How many PTOs does a regular employee have a year?"
query_vector = get_sentence_embedding(query)
query_vector = np.array(query_vector)

In [36]:
#Assigning average embedding and finding the closest subreddit -- create new col 'query_to_average' and then sort by the new column
db = db.assign(query_to_average = [cos_angle(query_vector, x)[0][0] for x in db['average_embedding']]).sort_values('query_to_average', ascending = False) 
db = db.reset_index().drop(columns='index') #resetting indices
db.head(10)

,subreddit,dataframe,average_embedding,query_to_average
0,FedEmployees,red...,"[-0.0045604230282108235, -0.006909224058766821...",0.869348
1,WalmartEmployees,r...,"[-0.008500097343943296, -0.00248342203242274, ...",0.863571
2,BestBuyWorkers,re...,"[-0.005873798435248916, -0.0051915133856321564...",0.860327
3,RiteAid,re...,"[-0.0054854856773563125, -0.002291048301678379...",0.857882
4,PaneraEmployees,re...,"[-0.006839217325948276, -0.0024444137202842757...",0.853538
5,cabincrewcareers,r...,"[0.00043171990228924506, -0.005137783052966276...",0.848919
6,TalesFromYourBank,r...,"[-0.005197360952537699, -0.006250292531688477,...",0.842512
7,Chase,r...,"[-0.011372712559842126, -0.0010401233618315545...",0.820753
8,KrakenSupport,r...,"[-0.0017264996006939439, -0.001783643988806693...",0.800380


In [37]:
#Assigning average embedding to each cluster and finding the closest clusters
db_cluster = db_cluster.assign(query_to_average = [cos_angle(query_vector, x)[0][0] for x in db_cluster['average_embedding']]).sort_values('query_to_average', ascending = False)
db_cluster = db_cluster.reset_index().drop(columns='index') #resetting indices
db_cluster.head(10)

,subreddit,dataframe,average_embedding,query_to_average
0,BestBuyWorkers-Cluster3,re...,"[-0.0025261466532921473, -0.007745908396475855...",0.880474
1,WalmartEmployees-Cluster0,r...,"[-0.005093428223200427, -0.000827104138358493,...",0.874014
2,RiteAid-Cluster1,re...,"[-0.0034636042534133145, -0.007548518882219773...",0.872972
3,FedEmployees-Cluster1,red...,"[-0.003005605609307194, -0.014883679618082091,...",0.866990
4,PaneraEmployees-Cluster0,re...,"[-0.002430848551148892, -0.004593826097116363,...",0.866774
5,FedEmployees-Cluster2,red...,"[-0.0019716562598081177, 0.00415739794698311, ...",0.862142
6,WalmartEmployees-Cluster1,r...,"[-0.01105996694646753, -0.0014071418133920317,...",0.857107
7,BestBuyWorkers-Cluster1,re...,"[-0.002131252957801869, -0.007079905009041493,...",0.856871
8,TalesFromYourBank-Cluster1,r...,"[-0.003341803076366174, -0.007956792389939725,...",0.849788
9,FedEmployees-Cluster3,red...,"[-0.008685157023137435, -0.004481811649621586,...",0.848829


In [43]:
df_bestmatch_nocluster = pd.concat([db.iloc[i].get('dataframe') for i in range(3)]) #top 3 subreddits
df_bestmatch_nocluster

,reddit_text,reddit_subreddit,textembedding
0,**Personal Investment Performance (PIP** - The...,FedEmployees,"[-0.02338474802672863, -0.0011061999248340726,..."
1,**[More Lifecycle Funds will be available July...,FedEmployees,"[0.009730241261422634, 0.012903322465717793, -..."
2,"/u/CH1EFK1NGD0M, I have found an error in your...",FedEmployees,"[-0.0019741507712751627, -0.02949591539800167,..."
3,1yr from date you started the position in that...,FedEmployees,"[-0.011592128314077854, 0.018938761204481125, ..."
4,21% gains,FedEmployees,"[-0.0038938489742577076, 0.027892155572772026,..."
...,...,...,...
5368,“Who are you to deny the return of the PS5 box...,BestBuyWorkers,"[-0.008801989257335663, -0.010793385095894337,..."
5369,“the narrative” Lmao or it’s just likely that ...,BestBuyWorkers,"[-0.008601270616054535, -0.021344924345612526,..."
5370,… yes? Are you one of the programmers did I hu...,BestBuyWorkers,"[0.02130361832678318, -0.017897486686706543, -..."
5371,🤣🤣 thank you for the advice. Mind elaborating ...,BestBuyWorkers,"[-0.00029987579910084605, 0.002727767685428261..."


In [45]:
df_bestmatch_cluster = pd.concat([db_cluster.iloc[i].get('dataframe') for i in range(3)]) #top 3 clusters
df_bestmatch_cluster

,reddit_text,reddit_subreddit,textembedding,Cluster ID
16,"""Anonymous"", sorry to give you the bad news bu...",BestBuyWorkers,"[-0.007506015244871378, -0.014071070589125156,...",3
50,. It’s also still annually when you would norm...,BestBuyWorkers,"[-0.012578140012919903, -0.013978547416627407,...",3
51,"...Shift-leads, how does the differential refl...",BestBuyWorkers,"[0.017350517213344574, 0.008666640147566795, -...",3
57,1 You’re a douchebag if you think people who w...,BestBuyWorkers,"[0.026971837505698204, 0.009699751622974873, -...",3
68,100% agree and I think the CIA position is nic...,BestBuyWorkers,"[-0.003729616291821003, -0.01914779655635357, ...",3
...,...,...,...,...
3654,why did they schedule it when you were away on...,RiteAid,"[-0.0024575316347181797, -0.002191046252846718...",1
3658,yeah just signed an offer and had to take one ...,RiteAid,"[-0.012678598053753376, -0.02642296813428402, ...",1
3660,yeah we didn’t get paid either,RiteAid,"[-0.006170068401843309, -0.011933591216802597,...",1
3668,yup the micromanaging has what’s annoyed me th...,RiteAid,"[0.013876966200768948, -0.023693732917308807, ...",1


In [46]:
#Without clustering, the runtime is over 1s
df_bestmatch_nocluster = df_bestmatch_nocluster.assign(cos_angles = [cos_angle(query_vector, np.array(x))[0][0] for x in df_bestmatch_nocluster['textembedding']]).sort_values('cos_angles', ascending = False)

In [47]:
df_bestmatch_nocluster.head(20)

,reddit_text,reddit_subreddit,textembedding,cos_angles
5678,"Not sure about PTO, but the most ppto you can ...",WalmartEmployees,"[-0.017012763768434525, 0.001875273184850812, ...",0.881989
301,"All associates earn PPTO, which is intended fo...",WalmartEmployees,"[-0.022059394046664238, -0.0017816615290939808...",0.879523
1619,Full timers generally earn pto faster. But mos...,WalmartEmployees,"[-0.008347814902663231, 0.010986099019646645, ...",0.879252
4867,Just went through onboarding (for Walmart dist...,WalmartEmployees,"[-0.0019015533616766334, -0.009029080159962177...",0.878480
2620,I get 21 mins of ppto earned per shift (10 hrs...,WalmartEmployees,"[0.003536654869094491, 0.007027565501630306, -...",0.877869
2275,It is like pto,BestBuyWorkers,"[-0.017847731709480286, -0.02621033973991871, ...",0.877719
8914,Yeah I'm aware of the benefits and pto. I just...,WalmartEmployees,"[0.004729983396828175, -0.006030685734003782, ...",0.876310
2811,Most places I've seen typically provide 0 PTO ...,BestBuyWorkers,"[-0.013062798418104649, -0.016060488298535347,...",0.875701
5655,Not everyone gets ppto at the same rate. You s...,WalmartEmployees,"[-0.008390921168029308, 0.01311011053621769, -...",0.874638
10041,"overtime, bonuses, PTO",WalmartEmployees,"[-0.0038837369065731764, -0.01123464573174715,...",0.873787


In [48]:
#By clustering, we cut down our runtown to under 1s 
df_bestmatch_cluster = df_bestmatch_cluster.assign(cos_angles = [cos_angle(query_vector, np.array(x))[0][0] for x in df_bestmatch_cluster['textembedding']]).sort_values('cos_angles', ascending = False)

In [49]:
df_bestmatch_cluster.head(20)

,reddit_text,reddit_subreddit,textembedding,Cluster ID,cos_angles
5678,"Not sure about PTO, but the most ppto you can ...",WalmartEmployees,"[-0.017012763768434525, 0.001875273184850812, ...",0,0.881989
301,"All associates earn PPTO, which is intended fo...",WalmartEmployees,"[-0.022059394046664238, -0.0017816615290939808...",0,0.879523
1619,Full timers generally earn pto faster. But mos...,WalmartEmployees,"[-0.008347814902663231, 0.010986099019646645, ...",0,0.879252
4867,Just went through onboarding (for Walmart dist...,WalmartEmployees,"[-0.0019015533616766334, -0.009029080159962177...",0,0.878480
2620,I get 21 mins of ppto earned per shift (10 hrs...,WalmartEmployees,"[0.003536654869094491, 0.007027565501630306, -...",0,0.877869
2275,It is like pto,BestBuyWorkers,"[-0.017847731709480286, -0.02621033973991871, ...",3,0.877719
8914,Yeah I'm aware of the benefits and pto. I just...,WalmartEmployees,"[0.004729983396828175, -0.006030685734003782, ...",0,0.876310
2811,Most places I've seen typically provide 0 PTO ...,BestBuyWorkers,"[-0.013062798418104649, -0.016060488298535347,...",3,0.875701
5655,Not everyone gets ppto at the same rate. You s...,WalmartEmployees,"[-0.008390921168029308, 0.01311011053621769, -...",0,0.874638
10041,"overtime, bonuses, PTO",WalmartEmployees,"[-0.0038837369065731764, -0.01123464573174715,...",0,0.873787


## 3. Improving performance via synthetic query generation and averaging 


In [50]:
#We generate synethetic queries to help improve the performance of our model 
input = tokenizer.encode(query, return_tensors='pt')

outputs = model.generate(
    input_ids=input,
    max_length=16,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=3)

print("Original query:")

print(query)

print("\nGenerated Synthetic Queries:")

for i in range(len(outputs)):
    q = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(f'{i + 1}: {q}')

Original query:
How many PTOs does a regular employee have a year?

Generated Synthetic Queries:
1: how many pto hours do employees earn
2: how many pto hours do regular employees get
3: how many hours of pto can an employee earn


In [51]:
chosen_query_vectors = [get_sentence_embedding(tokenizer.decode(x, skip_special_tokens=True)) for x in outputs]
chosen_query_vectors.append(query_vector)

In [52]:
avg_cos_angles = []

for x in df_bestmatch_cluster['textembedding']:
    avg_cos_angle = 0
    for v in chosen_query_vectors:
        avg_cos_angle += cos_angle(np.array(v), np.array(x))[0][0]
    avg_cos_angles.append(avg_cos_angle/len(chosen_query_vectors))
    

In [53]:
df_bestmatch_cluster = df_bestmatch_cluster.assign(avg_cos_angles = avg_cos_angles).sort_values('cos_angles', ascending = False)
df_bestmatch_by_avg = df_bestmatch_cluster.assign(avg_cos_angles = avg_cos_angles).sort_values('avg_cos_angles', ascending = False)

In [54]:
df_bestmatch_cluster.head(20)

,reddit_text,reddit_subreddit,textembedding,Cluster ID,cos_angles,avg_cos_angles
5678,"Not sure about PTO, but the most ppto you can ...",WalmartEmployees,"[-0.017012763768434525, 0.001875273184850812, ...",0,0.881989,0.909629
301,"All associates earn PPTO, which is intended fo...",WalmartEmployees,"[-0.022059394046664238, -0.0017816615290939808...",0,0.879523,0.891735
1619,Full timers generally earn pto faster. But mos...,WalmartEmployees,"[-0.008347814902663231, 0.010986099019646645, ...",0,0.879252,0.899054
4867,Just went through onboarding (for Walmart dist...,WalmartEmployees,"[-0.0019015533616766334, -0.009029080159962177...",0,0.878480,0.882403
2620,I get 21 mins of ppto earned per shift (10 hrs...,WalmartEmployees,"[0.003536654869094491, 0.007027565501630306, -...",0,0.877869,0.904211
2275,It is like pto,BestBuyWorkers,"[-0.017847731709480286, -0.02621033973991871, ...",3,0.877719,0.872764
8914,Yeah I'm aware of the benefits and pto. I just...,WalmartEmployees,"[0.004729983396828175, -0.006030685734003782, ...",0,0.876310,0.873622
2811,Most places I've seen typically provide 0 PTO ...,BestBuyWorkers,"[-0.013062798418104649, -0.016060488298535347,...",3,0.875701,0.868772
5655,Not everyone gets ppto at the same rate. You s...,WalmartEmployees,"[-0.008390921168029308, 0.01311011053621769, -...",0,0.874638,0.899730
10041,"overtime, bonuses, PTO",WalmartEmployees,"[-0.0038837369065731764, -0.01123464573174715,...",0,0.873787,0.888385


In [55]:
df_bestmatch_by_avg.head(20)

,reddit_text,reddit_subreddit,textembedding,Cluster ID,cos_angles,avg_cos_angles
5678,"Not sure about PTO, but the most ppto you can ...",WalmartEmployees,"[-0.017012763768434525, 0.001875273184850812, ...",0,0.881989,0.909629
2620,I get 21 mins of ppto earned per shift (10 hrs...,WalmartEmployees,"[0.003536654869094491, 0.007027565501630306, -...",0,0.877869,0.904211
5655,Not everyone gets ppto at the same rate. You s...,WalmartEmployees,"[-0.008390921168029308, 0.01311011053621769, -...",0,0.874638,0.899730
1619,Full timers generally earn pto faster. But mos...,WalmartEmployees,"[-0.008347814902663231, 0.010986099019646645, ...",0,0.879252,0.899054
301,"All associates earn PPTO, which is intended fo...",WalmartEmployees,"[-0.022059394046664238, -0.0017816615290939808...",0,0.879523,0.891735
1517,It varies greatly by state/location and store ...,RiteAid,"[0.0023047644644975662, 0.0036134349647909403,...",1,0.866713,0.888629
10041,"overtime, bonuses, PTO",WalmartEmployees,"[-0.0038837369065731764, -0.01123464573174715,...",0,0.873787,0.888385
80,1.00 = one hour.So you have almost an hour of ...,WalmartEmployees,"[0.010056194849312305, 0.01743551343679428, -0...",0,0.863413,0.885810
3324,I was just remembering earlier that it’s suppo...,WalmartEmployees,"[-0.00040522910421714187, 0.007721611764281988...",0,0.857598,0.885746
2276,It is not a month if pto bro when you first ge...,BestBuyWorkers,"[0.0026660107541829348, -0.008649304509162903,...",3,0.864908,0.884134


## 4. Feeding the ranked and re-ranked sentences to LLM

We introduce a function to generate a prompt for our LLM from top $k$ comments from ranked (or re-ranked comments):

In [56]:
def rag_prompt(df_col, k):
    information_to_feed = ""
    for n, text in zip(range(k), df_col):
        information_to_feed += f"{n+1}: " + text + "\n"
    # concatenate the first top k comments
    combined_information = f"\nQuery: {query}\n\nAnswer the above query by only using the following:\n\n{information_to_feed}\n\nLLM Response:"
    
    return combined_information

In [57]:
prompt_ranked = rag_prompt(df_bestmatch_cluster['reddit_text'], 5) # prompt generated from top 5 commenets ranked by cos sim to the original query
prompt_reranked = rag_prompt(df_bestmatch_by_avg['reddit_text'], 5) # prompt generated from top 5 commenets reranked by cos sim to the original query + alternative queries

**Warning**. To reproduce the following, one may need HuggingFace API Key (which is free for the purpose of this notebook).

In [58]:
auto_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below
auto_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.99s/it]


We write a function that produces a response from our LLM given a prompt.

In [59]:
def llm(prompt):
    input_ids = auto_tokenizer(prompt, return_tensors="pt")
    response = auto_model.generate(**input_ids, max_new_tokens=512)
    return auto_tokenizer.decode(response[0])

The following takes a while, but we believe that this can be improved by using a higher-performing LLM:

In [60]:
llm_answer1 = llm(prompt_ranked)
llm_answer2 = llm(prompt_reranked)

In [61]:
print("Using top 5 commenets ranked by cos sim to the original query \n\n", llm_answer1)

Using top 5 commenets ranked by cos sim to the original query 

 <bos>
Query: How many PTOs does a regular employee have a year?

Answer the above query by only using the following:

1: Not sure about PTO, but the most ppto you can earn in a year is 48 hours, with the exception of a couple of states which are unlimited by state law.
2: All associates earn PPTO, which is intended for emergencies (car won't start, you're sick, etcFull-time associates also earn PTO, which is intended for scheduled absences (doctor appointments, getting your drivers license renewed, vacations, etc. **Part-time associates do not earn PTO until they have been with the company for 3 years,** because part-time associates should be able to schedule their appointments and other errands on their days off.
3: Full timers generally earn pto faster. But most part timers actually earn ppto faster. For most associates in most states, ppto is 1 hour for every 30 hours worked. In most states the ppto accrual rate doesn’

In [62]:
print("Using top 5 commenets reranked by cos sim to the original query + alternative queries \n\n", llm_answer2)

Using top 5 commenets reranked by cos sim to the original query + alternative queries 

 <bos>
Query: How many PTOs does a regular employee have a year?

Answer the above query by only using the following:

1: Not sure about PTO, but the most ppto you can earn in a year is 48 hours, with the exception of a couple of states which are unlimited by state law.
2: I get 21 mins of ppto earned per shift (10 hrs and 19 mins of pto ... some states have different rules/laws but so I earn 84 minutes of ppto a week and 76 mins of regular
3: Not everyone gets ppto at the same rate. You start earning from day one and it’s available to use after 90 days. If you’re part time and less than 3 years, it’s 1 ppto hour for every 43.33 hours worked. Full timers, and part timers after 3 years, get 1 ppto hour for every 30 hours worked. Some states have no max. And if you work more than 2080 hours in a year then you will max out on pto
4: Full timers generally earn pto faster. But most part timers actually e

## 5. Evaluation of retrieval

Note that it is rather difficult to say which LLM responses are better. Moreover, we note that our goal is NOT to get the answer that is absolutely correct but a relevant one among the reddit comments that we put in. For example, the answer may change over time, unless we update the input comments.

Hence, we use use both of the LLM responses as ground truths and compare the top 50 retrievals from the two methods:
* Method 1: Naive RAG using cosine similairties against the original query
* Method 2: Not-so-naive RAG using average cosine similairties against multiple similar queries, including the original one

The following function separates the LLM response from an LLM answer:

In [63]:
def llm_answer_to_response(answer):
    start = answer.find("LLM Response:")
    end = answer.find("<eos>")
    return answer[start+len("LLM Response:")+1:end]

print(llm_answer_to_response(llm_answer1))
print(llm_answer_to_response(llm_answer2))

Regular employees typically earn 48 hours of paid time off per year, with the exception of a few states that have unlimited PTO by state law.
According to the passage, regular employees have a maximum of 48 hours of PTO per year.


In [64]:
truth_1 = llm_answer_to_response(llm_answer1)
truth_2 = llm_answer_to_response(llm_answer2)

In [65]:
vectors_1 = []
vectors_2 = []

for i in range(50):
    vectors_1.append(df_bestmatch_cluster.get('textembedding').iloc[i])
    vectors_2.append(df_bestmatch_by_avg.get('textembedding').iloc[i])

### Evaluation metric 1: cosine precision
The following is a function with which we evaluate the retrieval from each method. Let $\boldsymbol{t}_1$ and $\boldsymbol{t}_2$ be the truth vectors. For each vector $\boldsymbol{v}$ from a batch, the cosine similarities $\cos(\boldsymbol{t}_1, \boldsymbol{v})$ and $\cos(\boldsymbol{t}_2, \boldsymbol{v})$ are in the interval $[-1, 1]$, but in all of our examples, we know they are in $[0, 1]$. We simply take the average of the two to measure how truthful $\boldsymbol{v}$ is. Note that the closer the average is to $1$, the more truthful $\boldsymbol{v}$ is.

Recall the definition of **precision**:
$$\mathrm{Precision} := \frac{\mathrm{Relevant \ retrieved \ instances}}{\mathrm{All \ retrieved \ instances}}.$$

Given a batch $B$, we define the **cosine precision** as follows:

$$\mathrm{Cosine \ Precision \ of } \ B := \frac{1}{2|B|}\sum_{\boldsymbol{v} \in B}  (\cos(\boldsymbol{t}_1, \boldsymbol{v}) + \cos(\boldsymbol{t}_2, \boldsymbol{v}))$$

In [66]:
def cos_precision(batch, t_1, t_2):
    t_1 = np.array(t_1)
    t_2 = np.array(t_2)
    
    sum = 0

    for v in batch:
        v = np.array(v)
        sum += (cos_angle(t_1, v) + cos_angle(t_2, v))
    return sum / (2*len(batch))

t_1 = get_sentence_embedding(truth_1)
t_2 = get_sentence_embedding(truth_2)

In [67]:
cos_precision(vectors_1, t_1, t_2)

array([[0.86954928]])

In [68]:
cos_precision(vectors_2, t_1, t_2)

array([[0.8786808]])

### Evaluation metric 2: ranked cosine precision

The following is a function that evaluates not only the retrieval, but also evaluates the ranking for the retrieved contexts.

Assume we retrieved $K$ comments in the context, ranked as $B = (x_1, \ldots, x_K)$.

We call the **precision at rank $m$** the cosine precision for the truncated context $B_m := (x_1, \ldots, x_m)$. And the ranked cosine precision is the average of these precisions.

$$
\text{Ranked Cosine Precision of } B := \frac{1}{K} \sum_{m = 1}^{K} \text{Cosine Precision of } B_m.
$$

Under this measurement, those comments ranked higher in the retrieved context will have a higher impact to the precision.

In [69]:
def cos_rank_precision(batch, t_1, t_2):
    sum = 0

    for m in range(1, len(batch)+1):
        sum += cos_precision(batch[:m], t_1, t_2)

    return sum / len(batch)

In [70]:
cos_rank_precision(vectors_1, t_1, t_2)

array([[0.87982534]])

In [71]:
cos_rank_precision(vectors_2, t_1, t_2)

array([[0.88779141]])

## 6. Conclusion

As we have seen in the example above, our averaging method improves the overall retrieval better by getting rid of possibly unrelated retrieved data by comparisions with multiple similar queries to the original one. The LLM API took some time, but it is evident that any stronger LLM we use would only make this process faster.